# Calculate the inventory aging / slow moving items for given Dataset

In [13]:
# # Import different libraries

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import datetime as dt

# Import the inventory dataset

In [14]:
df = pd.read_csv('Inventory management.csv')
df.head()

,Serial_number,Received_date,Received_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name
0,1,2022-06-22,2022-06,DBFF2353E4794747,SKU-DBFF2353E4794747,7978,Furnitures,168,Delivered,Poland
1,2,2022-02-17,2022-02,2ADA2FA9DF8940D2,SKU-2ADA2FA9DF8940D2,2861,Electronics,146,Delivered,Holland
2,3,2022-04-28,2022-04,DB78C0CAAC754E8A,SKU-DB78C0CAAC754E8A,4300,Phones,248,Received,Holland
3,4,2022-07-12,2022-07,643029223A6D4030,SKU-643029223A6D4030,7422,Furnitures,156,Delivered,Italy
4,5,2022-11-20,2022-11,23019617DD11413F,SKU-23019617DD11413F,3196,Electronics,189,Delivered,England


In [15]:
# Chaneg the date columns format

df.Received_date= pd.to_datetime(df.Received_date)
df.Received_month= pd.to_datetime(df.Received_month)

# Show the total inventory value

In [16]:
Total_inventory = df.pivot_table(index='Item_Status', values='COGS', columns='Category', aggfunc='sum')
Total_inventory

Category,Computing,Electronics,Fashion,Furnitures,Phones
Item_Status,,,,,
Delivered,63738149,63807815,63959700,63750441,63931936
Packed,3951950,3995913,3965535,3954754,4011535
Received,3980690,3963475,3955521,3996459,3994522
Shipped,4020192,3949121,3972464,3984681,3997151
lost,4004250,4020157,4021721,4005045,3989162


# Get the total loss that needs to be written-off

In [17]:
Lost_items = df[df.Item_Status.isin(['lost'])].copy()
Lost_items.pivot_table(index='Country_Name', columns='Category', values='COGS', aggfunc='sum').round(2)

Category,Computing,Electronics,Fashion,Furnitures,Phones
Country_Name,,,,,
England,600671,615121,599300,595582,601544
France,398701,399187,405535,391220,393181
Germany,798099,794854,797259,806667,796912
Holland,682524,696247,695566,687110,672784
Italy,510031,518600,533994,516050,522079
Poland,201971,202623,199968,201160,206283
Portugal,205473,191336,199070,208538,205720
Spain,606780,602189,591029,598718,590659


# Get the ending inventory balance filtering by item status

In [18]:
inventory_balance = df[df['Item_Status'].isin(['Packed', 'Received', 'picked'])].copy()
inventory_balance.head()

,Serial_number,Received_date,Received_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name
2,3,2022-04-28,2022-04-01,DB78C0CAAC754E8A,SKU-DB78C0CAAC754E8A,4300,Phones,248,Received,Holland
29,30,2022-09-01,2022-09-01,B6EFE342FB7E43CF,SKU-B6EFE342FB7E43CF,88,Fashion,182,Packed,Portugal
32,33,2022-12-05,2022-12-01,5221F103980F40B5,SKU-5221F103980F40B5,9430,Computing,213,Packed,Italy
34,35,2022-06-27,2022-06-01,176FF5A3CCE24C10,SKU-176FF5A3CCE24C10,9208,Computing,171,Packed,Holland
41,42,2022-07-05,2022-07-01,4D065DAC7502499B,SKU-4D065DAC7502499B,2757,Electronics,127,Received,Germany


## Calculate the aging per days for the ending inventory balance as of 2022-12-31

In [20]:
inventory_balance['Aging']= inventory_balance['Received_date'].apply(lambda x: (dt.datetime(2022,12,31) - x).days)
inventory_balance.head()

,Serial_number,Received_date,Received_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name,Aging
2,3,2022-04-28,2022-04-01,DB78C0CAAC754E8A,SKU-DB78C0CAAC754E8A,4300,Phones,248,Received,Holland,247
29,30,2022-09-01,2022-09-01,B6EFE342FB7E43CF,SKU-B6EFE342FB7E43CF,88,Fashion,182,Packed,Portugal,121
32,33,2022-12-05,2022-12-01,5221F103980F40B5,SKU-5221F103980F40B5,9430,Computing,213,Packed,Italy,26
34,35,2022-06-27,2022-06-01,176FF5A3CCE24C10,SKU-176FF5A3CCE24C10,9208,Computing,171,Packed,Holland,187
41,42,2022-07-05,2022-07-01,4D065DAC7502499B,SKU-4D065DAC7502499B,2757,Electronics,127,Received,Germany,179


## Calculate the provision for the inventory assuming the company policy as below:

### - less than 90 days           = Zero provision
### - From 91 : 120 days        = 30% provision
### - From 121 : 180 days      = 60% provision
### - Over 180 days                = 100% provision


In [21]:
def a(inventory_balance):
    if inventory_balance['Aging'] <= 90:
        val = 0
    elif inventory_balance['Aging'] > 90 and inventory_balance['Aging'] <= 120:
        val = inventory_balance['COGS'] * .3
    elif inventory_balance['Aging'] > 120 and inventory_balance['Aging'] <= 180:
        val = inventory_balance['COGS'] * .6
    else:
        val = inventory_balance['COGS'] * 1
    return val

inventory_balance['Provision'] = inventory_balance.apply(a, axis=1)
inventory_balance.set_index('Serial_number').head()

,Received_date,Received_month,UID,SKU,Delivery note,Category,COGS,Item_Status,Country_Name,Aging,Provision
Serial_number,,,,,,,,,,,
3,2022-04-28,2022-04-01,DB78C0CAAC754E8A,SKU-DB78C0CAAC754E8A,4300,Phones,248,Received,Holland,247,248.0
30,2022-09-01,2022-09-01,B6EFE342FB7E43CF,SKU-B6EFE342FB7E43CF,88,Fashion,182,Packed,Portugal,121,109.2
33,2022-12-05,2022-12-01,5221F103980F40B5,SKU-5221F103980F40B5,9430,Computing,213,Packed,Italy,26,0.0
35,2022-06-27,2022-06-01,176FF5A3CCE24C10,SKU-176FF5A3CCE24C10,9208,Computing,171,Packed,Holland,187,171.0
42,2022-07-05,2022-07-01,4D065DAC7502499B,SKU-4D065DAC7502499B,2757,Electronics,127,Received,Germany,179,76.2


# Arrange the data

In [22]:
Provision = inventory_balance[['Received_date','Received_month','UID','SKU', 'Category','COGS'
                  ,'Item_Status','Country_Name', 'Aging', 'Provision']]
Provision.head()

,Received_date,Received_month,UID,SKU,Category,COGS,Item_Status,Country_Name,Aging,Provision
2,2022-04-28,2022-04-01,DB78C0CAAC754E8A,SKU-DB78C0CAAC754E8A,Phones,248,Received,Holland,247,248.0
29,2022-09-01,2022-09-01,B6EFE342FB7E43CF,SKU-B6EFE342FB7E43CF,Fashion,182,Packed,Portugal,121,109.2
32,2022-12-05,2022-12-01,5221F103980F40B5,SKU-5221F103980F40B5,Computing,213,Packed,Italy,26,0.0
34,2022-06-27,2022-06-01,176FF5A3CCE24C10,SKU-176FF5A3CCE24C10,Computing,171,Packed,Holland,187,171.0
41,2022-07-05,2022-07-01,4D065DAC7502499B,SKU-4D065DAC7502499B,Electronics,127,Received,Germany,179,76.2


# Show the provision for each Country per Category

In [23]:
country_provision = inventory_balance.pivot_table(index=['Country_Name'], columns=['Category'], values=['Provision'],aggfunc='sum')
country_provision

Provision                                            
Category     Computing Electronics   Fashion Furnitures     Phones
Country_Name                                                      
England       756993.1    742366.4  753773.5   766913.5   759055.6
France        503221.9    494040.4  490411.2   489487.5   493811.3
Germany       981777.4    978024.4  986256.9   986920.1  1014113.1
Holland       839295.1    879305.1  851237.5   841289.8   872209.1
Italy         642942.0    640839.1  647920.7   647629.8   648816.6
Poland        250600.1    260011.5  246853.2   234510.8   245996.6
Portugal      254100.8    257342.5  248806.2   246759.4   255210.3
Spain         755006.4    740453.0  744557.4   733043.0   741383.9